In [64]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [65]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [66]:

image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [67]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [68]:
#Problem 1:
#Introduce and tune L2 regularization for both logistic and neural network models. 
#Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. 
#In TensorFlow, you can compute the L2 loss for a tensor t using nn.l2_loss(t). 
#The right amount of regularization should improve your validation / test accuracy.

batch_size = 128
#Tuned manually. Adding L2 increased accuracy from 85.4 to 88.1
l2_weight = .005

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  l2 = l2_weight * tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

num_steps = 3001
#num_steps = 1

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #print(offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    #print(batch_data.shape)
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 33.179375
Minibatch accuracy: 7.8%
Validation accuracy: 13.0%
Minibatch loss at step 500: 1.881782
Minibatch accuracy: 78.1%
Validation accuracy: 79.9%
Minibatch loss at step 1000: 0.714091
Minibatch accuracy: 84.4%
Validation accuracy: 80.8%
Minibatch loss at step 1500: 0.559762
Minibatch accuracy: 85.9%
Validation accuracy: 81.2%
Minibatch loss at step 2000: 0.541843
Minibatch accuracy: 85.2%
Validation accuracy: 80.8%
Minibatch loss at step 2500: 0.769898
Minibatch accuracy: 79.7%
Validation accuracy: 78.5%
Minibatch loss at step 3000: 0.795392
Minibatch accuracy: 81.2%
Validation accuracy: 81.1%
Test accuracy: 88.1%


In [69]:
#Problem 2:
#Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

batch_size = 10
#Too small a batch size torpedos accuracy. Decreased from 88.1 to 78.0
l2_weight = .005

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  l2 = l2_weight * tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

num_steps = 3001
#num_steps = 1

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #print(offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    #print(batch_data.shape)
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 28.034941
Minibatch accuracy: 10.0%
Validation accuracy: 21.0%
Minibatch loss at step 500: 2.842019
Minibatch accuracy: 70.0%
Validation accuracy: 69.1%
Minibatch loss at step 1000: 2.081406
Minibatch accuracy: 80.0%
Validation accuracy: 72.8%
Minibatch loss at step 1500: 3.352981
Minibatch accuracy: 60.0%
Validation accuracy: 70.4%
Minibatch loss at step 2000: 2.020250
Minibatch accuracy: 60.0%
Validation accuracy: 69.4%
Minibatch loss at step 2500: 0.857258
Minibatch accuracy: 80.0%
Validation accuracy: 69.6%
Minibatch loss at step 3000: 1.429538
Minibatch accuracy: 50.0%
Validation accuracy: 68.0%
Test accuracy: 74.5%


In [79]:
#PROBLEM
#Introduce Dropout on the hidden layer of the neural network. 
#Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation 
#results would be stochastic as well. 
#TensorFlow provides nn.dropout() for that, but you have to make sure it's only inserted during training.
#What happens to our extreme overfitting case?

#accuracy seems to be even worse with small batch size and dropout turned on.
#batch_size = 10
batch_size = 128
num_hidden_nodes = 1024
#tuned manually. Applying dropout to relu increases accuracy from 86.6 to 88.6
dropout_probability =.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  b1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  relu = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1), dropout_probability)
  #relu = tf.nn.dropout(relu, dropout_probability)
  w2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))

  b2 = tf.Variable(tf.zeros([num_labels]))
  
  
  # Training computation.
  logits = tf.matmul(relu, w2) + b2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  #valid_prediction = tf.nn.softmax(
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
  test_prediction  = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 478.981171
Minibatch accuracy: 7.0%
Validation accuracy: 33.9%
Minibatch loss at step 500: 21.867357
Minibatch accuracy: 73.4%
Validation accuracy: 82.0%
Minibatch loss at step 1000: 16.102669
Minibatch accuracy: 75.8%
Validation accuracy: 82.4%
Minibatch loss at step 1500: 5.161777
Minibatch accuracy: 85.2%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 8.160447
Minibatch accuracy: 80.5%
Validation accuracy: 82.5%
Minibatch loss at step 2500: 9.603075
Minibatch accuracy: 71.9%
Validation accuracy: 81.6%
Minibatch loss at step 3000: 6.634416
Minibatch accuracy: 75.8%
Validation accuracy: 81.6%
Test accuracy: 89.1%


In [ ]:
#Problem 4
#Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is 97.1%.
#One avenue you can explore is to add multiple layers.
#Another one is to use learning rate decay

batch_size = 128
num_hidden_nodes = 1024
dropout_probability =.6
begin_learning_rate =.005
decay_rate = .96
l2_weight = .005

num_steps = 10001
#num_steps = 1


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
  
  # Variables.
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  b1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  relu = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1), dropout_probability)
  w2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))

  b2 = tf.Variable(tf.zeros([num_labels]))
  
  
  # Training computation.
  logits = tf.matmul(relu, w2) + b2
  l2_1 = l2_weight * tf.nn.l2_loss(w1)
  l2_2 = l2_weight * tf.nn.l2_loss(w2)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_1 + l2_2
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(begin_learning_rate, global_step, num_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
  test_prediction  = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1970.502808
Minibatch accuracy: 7.8%
Validation accuracy: 8.2%